<a href="https://colab.research.google.com/github/mulqueenr/DSWG_MethylationWorkshop/blob/main/DSWG_Methylation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the DSWG Methylation workshop!

I'll be running you through an vignette of amethyst processing, based heavily off the vignette for PBMC processing available on the amethyst github repo.

We will be processing from breast cancer cell line cells.

Our goals in this workshop are to:

- Get familiar with quality metrics of DNA methylation
- Cluster single cell methylomes to separate out the cell lines.
- Perform copy number calling to confirm our clusters.

## Library Installation

I prebuilt some R libraries that we can use to skip the install process. If you want to run the install yourself, the code is commented out below. You can fully ignore the commented out code below. All it is showing is that we need to install quite a few dependencies to get to the installation of two packages that will be the focus on this workshop:

- **amethyst**: single cell methylation processing
- **copykit**: single cell whole genome copy number calling

In [ ]:
#first lets install packages for the tutorial
#if (!requireNamespace("BiocManager", quietly = TRUE))
#install.packages("BiocManager")

#library("BiocManager")
#BiocManager::install(c("caret", "devtools", "data.table", "dplyr", "furrr", "future", "future.apply",
#  "ggplot2", "grDevices", "gridExtra", "igraph", "irlba", "janitor", "Matrix", "methods", "pheatmap",
#  "plotly", "plyr", "purrr", "randomForest", "rhdf5", "rtracklayer", "Rtsne", "scales", "stats", "stringr",
#  "tibble", "tidyr", "umap", "utils"))

#library("devtools")

#devtools::install_github("JinmiaoChenLab/Rphenograph")
#devtools::install_github("KrishnaswamyLab/MAGIC/Rmagic")
#devtools::install_github("TomKellyGenetics/leiden")
#devtools::install_github("lrylaarsdam/amethyst")
#devtools::install_github("navinlabcode/copykit")

To save ~30min of installation time, I prebuild the library and uploaded to a google drive folder. The following code downloads the libraries, and uncompresses them, so we can use them in our R session.

In [4]:
#download prebuilt R libraries
!gdown 1r2P-GZa2QukEl-i619yK7dqeoEhGjBNN
!tar -xf R_library.tar.gz #unpack the library
#download data
#!gdown


Downloading...
From (original): https://drive.google.com/uc?id=1r2P-GZa2QukEl-i619yK7dqeoEhGjBNN
From (redirected): https://drive.google.com/uc?id=1r2P-GZa2QukEl-i619yK7dqeoEhGjBNN&confirm=t&uuid=3f591552-096d-4863-a51e-9f99707b1e8e
To: /content/R_library.tar.gz
100% 600M/600M [00:04<00:00, 150MB/s]


In [ ]:
If you check your files for this colab session, you should see them populate with two folders:

- R_library.tar.gz : this is our downloaded compressed libraries
- content : this is our decompressed folder

Now all we have to do is tell R where to look for these libraries and we should be good to go!

To do this, we are going to append the folder location to an R environment variable called ".libPaths".

Following this we will load the two packages.

In [7]:
.libPaths(c("/content/content/library", .libPaths()))
library(amethyst)
library(copykit)



    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    Welcome to Amethyst! If you use Amethyst in your work, please cite our associated manuscript:
      Rylaarsdam, L.E. et al. Single-cell DNA methylation analysis tool Amethyst resolves distinct non-CG methylation patterns in human astrocytes and oligodendrocytes.
      Commun. Biol. 8, 1468 (2025). https://www.nature.com/articles/s42003-025-08859-2 


Attaching amethyst version 1.0.4.
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMean

In [9]:
%%R
?runCountReads()

File: /tmp/RtmpIqFP05/Rtxt24d5c8e3559
runCountReads             package:copykit              R Documentation



_A_l_i_g_n_s _t_h_e _r_e_a_d_s _f_r_o_m _t_h_e _B_A_M _f_i_l_e _t_o _t_h_e _v_a_r_i_a_b_l_e _b_i_n_n_i_n_g _p_i_p_e_l_i_n_e.



_D_e_s_c_r_i_p_t_i_o_n:



     runCountReads performs the variable binning (VarBin) algorithm to

     a series of BAM files resulting from short-read sequencing.



_U_s_a_g_e:



     runCountReads(

       dir,

       genome = c("hg38", "hg19"),

       resolution = c("220kb", "55kb", "110kb", "195kb", "280kb", "500kb", "1Mb", "2.8Mb"),

       remove_Y = FALSE,

       min_bincount = 10,

       is_paired_end = FALSE,

       BPPARAM = bpparam()

     )

     

_A_r_g_u_m_e_n_t_s:



     dir: A path for the directory containing BAM files from short-read

          sequencing.



  genome: Name of the genome assembly. Default: 'hg38'.



resolution: The resolution 